In [20]:

import tensorflow as tf
import numpy as np

from os.path import join as join_path

import pandas

from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix, classification_report
from sklearn.metrics import precision_recall_curve, f1_score
from sklearn import preprocessing


# import matplotlib as plt
# import matplotlib.pyplot as plt



# Design
Original data set has 2801 samples x 5072 features. This data were split into 70% train and 30% test. This process was repeated 7 times to create 7 different train-test splits.

For each split:
1. Create a random model
2. Evaluate the model on 35% data  with itest
3. Evaluate the model on 35% data + 35% pseudo-label data with itest
4. Evaluate the model on 35% data + high confident pseudo-label data with itest



Box plots for each model (1 box plot show the average accuracies of each model over the 7 train-test splits)

## 1. Read Files

+ file1 -
+ file2 -
+ file3 -

In [91]:
file1 = './data/seed2_35perc_train.csv'
file2='./data/seed2_70perc_SSLabel.csv'
file3='./data/seed2_SSLabel_HC.csv'
file4 = './data/seed2_holdOutTest.csv'



In [96]:
import scipy.io
import pandas

def read_Mat(filename):
    d = scipy.io.loadmat(filename)
    xtrain = d['xtrain']
    ytrain = [x[0][0].strip() for x in d['ytrain']]
    xitest = d['xitest']
    yitest = [x[0][0].strip() for x in d['yitest']]
    return(xtrain, ytrain, xitest, yitest)

def process_csv(filename):
    df = pandas.read_csv(filename, index_col=0)
    y = df.y
    X = df.drop(columns=['y'])
    return (X, y)

In [7]:
X35, y35 = preocess_csv(file1)
X70, y70 = process_csv(file2)
XHC, yHC = process_csv(file3)
xitest, yitest = process_csv(file4)

In [94]:
testd=pandas.read_csv(file1, index_col=0)

In [95]:
testd

,cg22054918,cg16476975,cg03940848,cg01517680,cg11536474,cg02809746,cg05649108,cg24375409,cg00295794,cg17129986,...,cg26874872,cg08393822,cg02099543,cg23291886,cg01703205,cg08703540,cg01295339,cg16721177,cg03032497,y
GSM2405316_10003886253_R06C02,0.620458,0.827531,0.725917,0.915672,0.931973,0.799222,0.813310,0.869353,0.917437,0.797108,...,0.102879,0.134548,0.890458,0.878794,0.337854,0.845982,0.766492,0.330316,0.851220,A IDH
GSM2404748_9769100008_R01C02,0.896388,0.759892,0.824178,0.872311,0.876098,0.784809,0.857543,0.855149,0.913162,0.810768,...,0.111749,0.106031,0.910376,0.427017,0.520547,0.770784,0.782599,0.863862,0.763572,A IDH
GSM2404751_9769100024_R02C01,0.929032,0.546564,0.864100,0.886982,0.900447,0.774137,0.894101,0.858779,0.882877,0.814876,...,0.103359,0.193778,0.937631,0.830720,0.817933,0.850669,0.858232,0.293305,0.882825,A IDH
GSM2405002_9969477022_R03C01,0.951399,0.597364,0.916993,0.657680,0.930617,0.934408,0.947559,0.955195,0.951609,0.917606,...,0.122598,0.185687,0.934678,0.901829,0.860300,0.881294,0.637854,0.182095,0.890568,A IDH
GSM2405084_9969477108_R02C02,0.798488,0.679394,0.792001,0.869149,0.850188,0.765071,0.841396,0.883066,0.860462,0.825742,...,0.095813,0.220581,0.901222,0.747438,0.551964,0.863224,0.763140,0.797105,0.842421,A IDH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM2403628_8622007189_R06C01,0.041818,0.047810,0.032837,0.061170,0.818647,0.043171,0.056462,0.032441,0.599684,0.637017,...,0.779651,0.162710,0.105015,0.283996,0.603156,0.027712,0.061334,0.087986,0.243316,"SUBEPN, ST"
GSM2403198_6929806038_R04C02,0.531490,0.027112,0.028448,0.047980,0.884155,0.020599,0.024129,0.022131,0.673599,0.024955,...,0.919554,0.131408,0.038156,0.149619,0.876234,0.028990,0.029467,0.353489,0.214306,"SUBEPN, ST"
GSM2403207_6929806046_R04C02,0.032064,0.021988,0.024431,0.107715,0.200298,0.021182,0.035996,0.028614,0.111856,0.030046,...,0.635093,0.137849,0.051868,0.110168,0.517920,0.030511,0.026732,0.219361,0.309451,"SUBEPN, ST"
GSM2402991_6929689149_R02C02,0.400427,0.020836,0.020135,0.048822,0.257915,0.019289,0.020282,0.027842,0.039483,0.055963,...,0.919059,0.125473,0.035442,0.167203,0.884822,0.026775,0.018601,0.071085,0.189290,"SUBEPN, ST"


# 2. Define results

In [61]:
import pandas

result_df = pandas.DataFrame(data=[], columns=['Seed','Dataset','Validation','Metric','Value'])

def append_result(p, df):
    s = pandas.Series(p, index=df.columns)
    return df.append(s, ignore_index=True)
    
# result_df = append_result([2, 35, 'balanced_acc', b_acc], result_df)
result_df

,Seed,Dataset,Validation,Metric,Value


## 1. Create an RF model with 5 repeated 3-fold CV

In [67]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate, RepeatedStratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, balanced_accuracy_score
import pandas
import numpy as np

def apply_CV(mod, cv, X, y):
    return cross_validate(mod, X, y, cv=cv, scoring=['balanced_accuracy', 'f1_weighted', 'roc_auc_ovr_weighted'])

def append_cv_results(df, r, seed_name, dataset_name):
    df = append_result([seed_name, dataset_name, 'cross_val', 'balanced_acc', r['test_balanced_accuracy'].mean().round(2)], df)
    df = append_result([seed_name, dataset_name, 'cross_val', 'f1', r['test_f1_weighted'].mean().round(2)], df)
    df = append_result([seed_name, dataset_name, 'cross_val', 'roc_auc_ovr', r['test_roc_auc_ovr_weighted'].mean().round(2)], df)
    return df

def evaluate_against_test_set(mod, x_train, y_train, x_test, y_test):
    mod.fit(x_train, y_train)
    y_pred = mod.predict(x_test)
    prec = precision_score(y_test,y_pred, average='weighted', zero_division=0).round(3)
    rec = recall_score(y_test, y_pred,  average='weighted', zero_division=0).round(3)
    acc = accuracy_score(y_pred, y_test).round(3)
    b_acc = balanced_accuracy_score(y_pred, y_test).round(3)
    return (prec, rec, acc, b_acc)

def append_testset_results(df, r, seed_name, dataset_name):
    df = append_result([seed_name, dataset_name, 'vs_testset', 'prec', r[0]], df)
    df = append_result([seed_name, dataset_name, 'vs_testset', 'rec', r[1]], df)
    df = append_result([seed_name, dataset_name, 'vs_testset', 'acc', r[2]], df)
    df = append_result([seed_name, dataset_name, 'vs_testset', 'balanced_acc', r[3]], df)
    return df


In [56]:
model = RandomForestClassifier(n_estimators = 50, max_depth=100)
cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=5)

## 2. Build a RF with 35% data

In [57]:
result35 = apply_CV(model, cv, X35, y35)

In [58]:
result35

{'fit_time': array([2.60769081, 2.65561295, 2.58696389, 2.6937151 , 2.58895373,
        2.64200997, 2.64927602, 2.70846581, 2.799541  , 2.62039399,
        2.63421535, 2.66131783, 2.71606183, 2.6953001 , 2.64592719]),
 'score_time': array([0.05341125, 0.05587101, 0.0539012 , 0.05500889, 0.05277419,
        0.06107521, 0.05360484, 0.05768824, 0.05497408, 0.06454682,
        0.05538082, 0.06257105, 0.05113029, 0.05491877, 0.07017612]),
 'test_balanced_accuracy': array([0.67960928, 0.72683368, 0.74066601, 0.76023037, 0.68634005,
        0.66856486, 0.73323637, 0.71333944, 0.71471374, 0.6952109 ,
        0.72700157, 0.66321093, 0.72773648, 0.72978922, 0.72526565]),
 'test_f1_weighted': array([0.79646047, 0.79550521, 0.82172496, 0.81362922, 0.79579504,
        0.78548876, 0.82345212, 0.79314734, 0.8093636 , 0.79301866,
        0.8048172 , 0.77829634, 0.80956983, 0.81160327, 0.80151355]),
 'test_roc_auc_ovr_weighted': array([0.99357942, 0.9968408 , 0.99180819, 0.99405429, 0.99671103,
       

In [64]:
result_df = append_cv_results(result_df, result35, 2, 35)

In [65]:
result_df

,Seed,Dataset,Validation,Metric,Value
0,2,35,cross_val,balanced_acc,0.71
1,2,35,cross_val,f1,0.80
2,2,35,cross_val,roc_auc_ovr,0.99


## 3. Validate on independent hold-out test set

In [68]:
result_ts_35 = evaluate_against_test_set(model, X35, y35, xitest, yitest)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1814: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')


In [70]:
result_ts_35

(0.857, 0.878, 0.878, 0.892)

In [74]:
result_df = append_testset_results(result_df, result_ts_35, 2, 35)

In [75]:
result_df

,Seed,Dataset,Validation,Metric,Value
0,2,35,cross_val,balanced_acc,0.710
1,2,35,cross_val,f1,0.800
2,2,35,cross_val,roc_auc_ovr,0.990
3,2,35,vs_testset,prec,0.857
4,2,35,vs_testset,rec,0.878
5,2,35,vs_testset,acc,0.878
6,2,35,vs_testset,balanced_acc,0.892


## 4. Build a RF with 70% data

In [76]:
result70 = apply_CV(model, cv, X70, y70)

In [77]:
result_df

,Seed,Dataset,Validation,Metric,Value
0,2,35,cross_val,balanced_acc,0.710
1,2,35,cross_val,f1,0.800
2,2,35,cross_val,roc_auc_ovr,0.990
3,2,35,vs_testset,prec,0.857
4,2,35,vs_testset,rec,0.878
5,2,35,vs_testset,acc,0.878
6,2,35,vs_testset,balanced_acc,0.892


In [79]:
result_df = append_cv_results(result_df, result70, 2, 70)

In [80]:
result_df

,Seed,Dataset,Validation,Metric,Value
0,2,35,cross_val,balanced_acc,0.710
1,2,35,cross_val,f1,0.800
2,2,35,cross_val,roc_auc_ovr,0.990
3,2,35,vs_testset,prec,0.857
4,2,35,vs_testset,rec,0.878
5,2,35,vs_testset,acc,0.878
6,2,35,vs_testset,balanced_acc,0.892
7,2,70,cross_val,balanced_acc,0.840
8,2,70,cross_val,f1,0.900
9,2,70,cross_val,roc_auc_ovr,1.000


In [81]:
result_ts_70 = evaluate_against_test_set(model, X70, y70, xitest, yitest)
result_ts_70

/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1814: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')


(0.929, 0.933, 0.933, 0.954)

In [83]:
result_df = append_testset_results(result_df, result_ts_70, 2, 70)
result_df

,Seed,Dataset,Validation,Metric,Value
0,2,35,cross_val,balanced_acc,0.710
1,2,35,cross_val,f1,0.800
2,2,35,cross_val,roc_auc_ovr,0.990
3,2,35,vs_testset,prec,0.857
4,2,35,vs_testset,rec,0.878
5,2,35,vs_testset,acc,0.878
6,2,35,vs_testset,balanced_acc,0.892
7,2,70,cross_val,balanced_acc,0.840
8,2,70,cross_val,f1,0.900
9,2,70,cross_val,roc_auc_ovr,1.000


## 5. Build a RF with HC pseudo-labeled data

In [84]:
resultHC = apply_CV(model, cv, XHC, yHC)

In [85]:
result_ts_HC = evaluate_against_test_set(model, XHC, yHC, xitest, yitest)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1814: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')


In [88]:
result_df = append_cv_results(result_df, resultHC, 2, 'HC')

In [89]:
result_df = append_testset_results(result_df, result_ts_HC, 2, 'HC')

In [90]:
result_df

,Seed,Dataset,Validation,Metric,Value
0,2,35,cross_val,balanced_acc,0.710
1,2,35,cross_val,f1,0.800
2,2,35,cross_val,roc_auc_ovr,0.990
3,2,35,vs_testset,prec,0.857
4,2,35,vs_testset,rec,0.878
5,2,35,vs_testset,acc,0.878
6,2,35,vs_testset,balanced_acc,0.892
7,2,70,cross_val,balanced_acc,0.840
8,2,70,cross_val,f1,0.900
9,2,70,cross_val,roc_auc_ovr,1.000
